In [ ]:
# ----------------------------
# Manipulación de datos
# ----------------------------
import numpy as np
from scipy import stats
import missingno as msno
import polars as pl
import kagglehub

# ----------------------------
# Visualización
# ----------------------------
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
from plotly.subplots import make_subplots

# ----------------------------
# Preprocesamiento y reducción de dimensionalidad
# ----------------------------
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

# ----------------------------
# Modelado
# ----------------------------
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# ----------------------------
# Producción / App
# ----------------------------
import streamlit as st
import os

print("¡Todo funciona perfectamente! 🚀")

¡Todo funciona perfectamente! 🚀


In [ ]:
import os

# Set credentials BEFORE importing the KaggleApi
os.environ['KAGGLE_USERNAME'] = 'jabeichbenavides123'  # e.g., 'johnsmith'
os.environ['KAGGLE_KEY'] = 'dc9e4472d88a97944f31b130d4134a0'            # e.g., '8a1b2c3d4e5f6g7h8i9j0'

# Now import and authenticate
from kaggle.api.kaggle_api_extended import KaggleApi
api = KaggleApi()
api.authenticate()  # This should now work

print("Authentication successful!")


Authentication successful!


In [105]:

# Ruta donde se descargará y descomprimirá el dataset
data_path = r"C:\Users\Coder\Documents\REGRESIONES\F5ProjectV_ProblemaDeRegresion\data"

# Descargar y descomprimir, reemplazando si ya existe
api.dataset_download_files(
    "abdulmalik1518/cars-datasets-2025",
    path=data_path,
    unzip=True,
    force=True  # <-- Sobrescribe si ya existe
)

# Detectar automáticamente el CSV dentro de la carpeta
csv_files = [f for f in os.listdir(data_path) if f.lower().endswith(".csv")]
if not csv_files:
    raise FileNotFoundError("No se encontró ningún archivo CSV en la carpeta de destino.")
csv_file = os.path.join(data_path, csv_files[0])

# Leer CSV con Polars
df = pl.read_csv(csv_file, encoding="latin1")

# Mostrar primeras filas
print(df.head())


Dataset URL: https://www.kaggle.com/datasets/abdulmalik1518/cars-datasets-2025
shape: (5, 11)
┌────────────┬────────────┬────────────┬───────────┬───┬───────────┬───────────┬───────┬───────────┐
│ Company    ┆ Cars Names ┆ Engines    ┆ CC/Batter ┆ … ┆ Cars      ┆ Fuel      ┆ Seats ┆ Torque    │
│ Names      ┆ ---        ┆ ---        ┆ y         ┆   ┆ Prices    ┆ Types     ┆ ---   ┆ ---       │
│ ---        ┆ str        ┆ str        ┆ Capacity  ┆   ┆ ---       ┆ ---       ┆ str   ┆ str       │
│ str        ┆            ┆            ┆ ---       ┆   ┆ str       ┆ str       ┆       ┆           │
│            ┆            ┆            ┆ str       ┆   ┆           ┆           ┆       ┆           │
╞════════════╪════════════╪════════════╪═══════════╪═══╪═══════════╪═══════════╪═══════╪═══════════╡
│ FERRARI    ┆ SF90       ┆ V8         ┆ 3990 cc   ┆ … ┆ $1,100,00 ┆ plug in   ┆ 2     ┆ 800 Nm    │
│            ┆ STRADALE   ┆            ┆           ┆   ┆ 0         ┆ hyrbrid   ┆       ┆          

In [106]:
# CAMBIO DE NOMBRES DE VARIABLES

# 1. Renombrar columnas para mayor claridad y consistencia
df = df.rename({
    "Company Names": "Brands",                       # Nombre de la marca
    "Cars Names": "Model",                           # Nombre del modelo
    "CC/Battery Capacity": "Engine_capacity_in_cc", # Capacidad del motor o batería
    "HorsePower": "HorsePower_in_HP",               # Potencia en caballos de fuerza
    "Total Speed": "Max_speed_in_km/h",             # Velocidad máxima en km/h
    "Performance(0 - 100 )KM/H": "Time_to_100kmph_sec", # Tiempo de 0 a 100 km/h en segundos
    "Cars Prices": "Price_$",                        # Precio en dólares
    "Fuel Types": "Fuel",                             # Tipo de combustible
    "Torque": "Torque_in_Nm"                          # Torque en Newton-metros
})

# 2. Crear una columna "id" como identificador único para cada fila
# pl.arange(0, pl.len()) genera números desde 0 hasta el número de filas
# .alias("id") le asigna el nombre "id" a esta nueva columna
df = df.with_columns(
    pl.arange(0, pl.len()).alias("id")
)

# 3. Seleccionar y reordenar solo las columnas necesarias
# Esto limpia el DataFrame y garantiza que tenga las columnas que nos interesan
df = df.select([
    "id",
    "Brands",                # Marca del auto
    "Model",                 # Modelo del auto
    "Engines",               # Tipo de motor
    "Engine_capacity_in_cc", # Capacidad del motor/batería
    "HorsePower_in_HP",      # Potencia
    "Max_speed_in_km/h",     # Velocidad máxima
    "Time_to_100kmph_sec",   # Tiempo de aceleración 0-100 km/h
    "Price_$",               # Precio
    "Fuel",                  # Tipo de combustible
    "Seats",                 # Número de asientos
    "Torque_in_Nm"           # Torque
])



In [107]:
# Descripciones de Datos iniciales
print("\nResumen de datos:")
print(df.describe())

# Valores nulos
print("\nValores nulos por columna:")
print(df.null_count())

# Número de valores únicos por columna
n_unique = {col: df[col].n_unique() for col in df.columns}
print("\nValores únicos por columna:")
for col, count in n_unique.items():
    print(f"{col}: {count}")


Resumen de datos:
shape: (9, 13)
┌────────────┬────────────┬────────┬────────────┬───┬────────────┬────────────┬───────┬────────────┐
│ statistic  ┆ id         ┆ Brands ┆ Model      ┆ … ┆ Price_$    ┆ Fuel       ┆ Seats ┆ Torque_in_ │
│ ---        ┆ ---        ┆ ---    ┆ ---        ┆   ┆ ---        ┆ ---        ┆ ---   ┆ Nm         │
│ str        ┆ f64        ┆ str    ┆ str        ┆   ┆ str        ┆ str        ┆ str   ┆ ---        │
│            ┆            ┆        ┆            ┆   ┆            ┆            ┆       ┆ str        │
╞════════════╪════════════╪════════╪════════════╪═══╪════════════╪════════════╪═══════╪════════════╡
│ count      ┆ 1218.0     ┆ 1218   ┆ 1218       ┆ … ┆ 1218       ┆ 1218       ┆ 1218  ┆ 1218       │
│ null_count ┆ 0.0        ┆ 0      ┆ 0          ┆ … ┆ 0          ┆ 0          ┆ 0     ┆ 0          │
│ mean       ┆ 608.5      ┆ null   ┆ null       ┆ … ┆ null       ┆ null       ┆ null  ┆ null       │
│ std        ┆ 351.750622 ┆ null   ┆ null       ┆ … ┆ nul

##1. Resumen de datos

El bloque df.describe() genera un resumen estadístico del DataFrame. Muestra información como el número de filas (count), la cantidad de valores nulos (null_count), medidas estadísticas básicas para columnas numéricas (mean, std, min, 25%, 50%, 75%, max) y los tipos de datos de cada columna (f64, str, etc.). En este caso, la tabla indica que hay 1218 filas y 13 columnas, con columnas numéricas como id que tienen estadísticas completas, y columnas de texto (Brands, Model) que muestran conteos y valores extremos. Esto permite identificar rápidamente la estructura, posibles inconsistencias y rangos de valores en el dataset.

##2. Valores nulos por columna

El bloque df.null_count() calcula la cantidad de valores nulos en cada columna del DataFrame. La salida indica que todas las columnas tienen cero valores nulos, lo que significa que el dataset está completo y no requiere imputación de datos. Cada columna se representa con su tipo (u32 para enteros no negativos) y el conteo de nulos por fila. Este análisis es fundamental en la limpieza de datos, ya que los valores faltantes pueden afectar cálculos estadísticos, modelado predictivo o agregaciones, y aquí confirma que no hay problemas de datos faltantes en ninguna columna.

##3. Valores únicos por columna

Este bloque calcula la cantidad de valores únicos en cada columna, mostrando diversidad y cardinalidad de los datos. Por ejemplo, id tiene 1218 valores únicos (uno por fila), Brands tiene 37 marcas diferentes y Model 1201 modelos distintos. Columnas numéricas como HorsePower_in_HP y Engine_capacity_in_cc tienen cientos de valores únicos, mientras que variables categóricas como Fuel y Seats tienen pocos valores distintos. Esta información es útil para identificar columnas categóricas frente a continuas, detectar redundancias y entender la granularidad del dataset, lo que ayuda a decidir transformaciones, agrupaciones o codificación en futuros análisis.

In [108]:
# Forma alternativa de inspeccionar el esquema
for name, dtype in df.schema.items():
    print(f"{name}: {dtype}")

id: Int64
Brands: String
Model: String
Engines: String
Engine_capacity_in_cc: String
HorsePower_in_HP: String
Max_speed_in_km/h: String
Time_to_100kmph_sec: String
Price_$: String
Fuel: String
Seats: String
Torque_in_Nm: String


Este bloque describe el esquema de un DataFrame, indicando el tipo de datos de cada columna, lo cual es fundamental para análisis y limpieza de datos.

**id:** Int64 es un identificador único numérico para cada fila, útil para referenciar registros y ordenar datos.

Columnas como **Brands**, **Model**, **Engines**, **Fuel** son de **tipo String**, almacenando texto que describe **marca**, **modelo**, **tipo de motor** y **combustible**.

Columnas de características como **Engine_capacity_in_cc, HorsePower_in_HP, Max_speed_in_km/h, Time_to_100kmph_sec, Price_$, Seats y Torque_in_Nm** también son String, por los formatos mixtos o símbolos.

Conocer estos tipos permite planificar conversión a numérico, limpieza y análisis estadístico correcto del dataset.

In [109]:
print("\n" + "="*60)
print("ANÁLISIS DE VALORES ÚNICOS POR COLUMNA")
print("="*60)

for col in df.columns:
    print(f"\nColumna: {col}")

    # Número de valores únicos
    unique_count = df[col].n_unique()
    print(f"Número de valores únicos: {unique_count}")

    # Obtener valores únicos y ordenarlos con manejo de None
    unique_values = df[col].unique().to_list()
    unique_values.sort(key=lambda x: str(x))  # evita error con None

    if unique_count <= 50:
        print("Valores únicos:")
        print(unique_values)
    else:
        print("Algunos valores únicos (muestra):")
        print(unique_values[:20])
        print(f"... y {unique_count - 20} valores más")

    # Valor más frecuente
    if unique_count > 0:
        most_frequent = df[col].mode().to_list()[0]
        print(f"Valor más frecuente: {most_frequent}")
    else:
        print("No hay valores en esta columna.")

    print("-" * 60)


ANÁLISIS DE VALORES ÚNICOS POR COLUMNA

Columna: id
Número de valores únicos: 1218
Algunos valores únicos (muestra):
[0, 1, 10, 100, 1000, 1001, 1002, 1003, 1004, 1005, 1006, 1007, 1008, 1009, 101, 1010, 1011, 1012, 1013, 1014]
... y 1198 valores más
Valor más frecuente: 0
------------------------------------------------------------

Columna: Brands
Número de valores únicos: 37
Valores únicos:
['ASTON MARTIN', 'AUDI', 'Acura', 'BENTLEY', 'BMW', 'Bugatti', 'Cadillac', 'Chevrolet', 'FERRARI', 'Ford', 'GMC', 'HONDA', 'HYUNDAI', 'Jaguar Land Rover', 'Jeep', 'KIA', 'KIA  ', 'Kia', 'LAMBORGHINI', 'MAHINDRA', 'MARUTI SUZUKI', 'MERCEDES', 'Mazda', 'Mitsubishi', 'NISSAN', 'Nissan', 'Peugeot', 'Porsche', 'ROLLS ROYCE', 'ROLLS ROYCE ', 'TOYOTA', 'Tata Motors', 'Tesla', 'Toyota', 'VOLVO', 'Volkswagen', 'Volvo']
Valor más frecuente: Nissan
------------------------------------------------------------

Columna: Model
Número de valores únicos: 1201
Algunos valores únicos (muestra):
[' GT 63 S', '100N

#Limpieza

Se realiza una limpieza de datos para garantizar que la base de datos sea funcional y consistente, lo que facilita el análisis y el trabajo posterior. Esta etapa incluye:

Eliminación de valores nulos o inconsistentes: se revisan columnas importantes y se gestionan los registros incompletos para evitar errores en el análisis.

Estandarización de nombres y formatos: se normalizan nombres de columnas, unidades de medida y formatos de texto para mantener la coherencia.

Filtrado de datos irrelevantes: se eliminan columnas o registros que no aportan valor al análisis.

Creación de identificadores únicos: se generan columnas de ID para cada registro, facilitando la referencia y manipulación de los datos.

El objetivo final es tener un conjunto de datos limpio, organizado y listo para procesos de transformación, análisis o visualización de manera eficiente y confiable.

## Modificacion Columna: Brands

In [110]:
df = df.with_columns(
    (
        pl.col("Brands").str.slice(0, 1).str.to_uppercase() +
        pl.col("Brands").str.slice(1).str.to_lowercase()
    ).alias("Brands")
)

## Modificacion Columna: Engine Capacity in CC



In [111]:
df = df.with_columns(
    pl.col("Engine_capacity_in_cc")
    .cast(pl.Utf8)                        # Convertir a string primero
    .str.replace_all(r"[Cc]{1,2}", "")    # Eliminar "cc" o "CC"
    .str.replace_all(r",", "")             # Eliminar comas
    .str.replace_all(r"\s*\+.*", "")      # Eliminar texto adicional
    .str.strip_chars()                     # Limpiar espacios
    .cast(pl.Int32, strict=False)          # Convertir a Int32, valores inválidos → null
    .fill_null(0)                          # Reemplazar nulos con 0
    .alias("Engine_capacity_in_cc")

)

# Verificar nulos
null_count = df["Engine_capacity_in_cc"].null_count()
print(f"Valores no convertibles: {null_count}")

Valores no convertibles: 0


## Modificacion Columna: HorsePower in HP

In [112]:
# Extraer ambos números potenciales en una sola operación
df = df.with_columns([
    pl.col("HorsePower_in_HP")
    .str.extract_groups(r"(?P<first>\d+)\s*(?:-\s*(?P<second>\d+))?\s*hp")
    .struct.rename_fields(["HorsePower_in_HP", "HorsePower_in_HP_2"])
    .alias("temp")
])

# Expandir la estructura y procesar los valores
df = df.with_columns([
    pl.col("temp").struct.field("HorsePower_in_HP").cast(pl.Int32),
    pl.col("temp").struct.field("HorsePower_in_HP_2").cast(pl.Int32)
]).drop("temp")

# Si el segundo valor es nulo, usar el primero
df = df.with_columns(
    pl.coalesce(pl.col("HorsePower_in_HP_2"), pl.col("HorsePower_in_HP")).alias("HorsePower_in_HP_2")
)

# Construir nuevo orden de columnas
cols = df.columns
cols.remove("HorsePower_in_HP_2")              # quitar 'id' de su posición actual
cols.insert(6, "HorsePower_in_HP_2")           # insertar 'id' en la posición 7 (índice 6)

# Reordenar
df = df.select(cols)

## Modificacion Columna: Price $

In [113]:
# Crear columnas Price_Min y Price_Max a partir de Price_$
df = df.with_columns([

    # Extraer el primer número (mínimo o único valor)
    pl.col("Price_$")
      .str.strip_chars()                              # quitar espacios extras
      .str.replace_all(r"[\$,]", "", literal=False)   # quitar $ y ,
      .str.extract(r"(\d+)")                          # tomar primer número
      .cast(pl.Float32)
      .alias("Price_Min"),

    # Extraer el segundo número si hay rango
    pl.col("Price_$")
      .str.strip_chars()
      .str.replace_all(r"[\$,]", "", literal=False)   # quitar $ y ,
      .str.extract(r"\d+\s*-\s*(\d+)")                # capturar segundo número
      .cast(pl.Float32)
      .alias("Price_Max")
])

# Si no hay rango, usar Price_Min como Price_Max
df = df.with_columns(
    pl.when(pl.col("Price_Max").is_null())
      .then(pl.col("Price_Min"))
      .otherwise(pl.col("Price_Max"))
      .alias("Price_Max")
)

# Eliminar la columna original de texto
df = df.drop("Price_$")

# Construir nuevo orden de columnas
cols = df.columns
cols.remove("Price_Min")              # quitar 'id' de su posición actual
cols.insert(9, "Price_Min")           # insertar 'id' en la posición 10 (índice 9)
cols.remove("Price_Max")              # quitar 'id' de su posición actual
cols.insert(10, "Price_Max")           # insertar 'id' en la posición 10 (índice 11)

# Reordenar
df = df.select(cols)


## Modificacion Columna: Max speed in km/h

In [114]:
df = df.with_columns([
    pl.col("Max_speed_in_km/h")
    .str.extract(r"(\d+)")  # Captura el número
    .cast(pl.Float32, strict=False)          # Convertir a entero
    .alias("Max_speed_in_km/h")
])

## Modificacion Columna: Time to 100kmph sec


In [115]:
df = df.with_columns([
    pl.col("Time_to_100kmph_sec")
    .str.extract(r"(\d+(?:\.\d+)?)")  # Captura el primer número decimal
    .cast(pl.Float32, strict=False)    # Convertir a float, valores inválidos → null
    .alias("Time_to_100kmph_sec")
])

## Modificacion Columna: Seats

In [116]:
df = df.with_columns([
    pl.col("Seats")
    .str.extract(r"(\d+(?:\.\d+)?)")  # Captura el primer número decimal
    .cast(pl.Int32, strict=False)    # Convertir a float, valores inválidos → null
    .alias("Seats")
])

## Modificacion Columna: Torque in Nm

In [117]:
df = df.with_columns([
    # Primer número (valor mínimo)
    pl.col("Torque_in_Nm")
    .str.extract(r"(\d+)")           # Captura el primer número
    .cast(pl.Int32)
    .alias("Torque_in_Nm"),

    # Segundo número si existe rango
    pl.col("Torque_in_Nm")
    .str.extract(r"\d+\s*-\s*(\d+)")  # Captura segundo número después de "-"
    .cast(pl.Int32)
    .alias("Torque_in_Nm_2_temp")
])

# Reemplazar null por el primer número si no hay rango
df = df.with_columns(
    pl.when(pl.col("Torque_in_Nm_2_temp").is_null())
      .then(pl.col("Torque_in_Nm"))
      .otherwise(pl.col("Torque_in_Nm_2_temp"))
      .alias("Torque_in_Nm_2")
)

# Eliminar columna temporal
df = df.drop("Torque_in_Nm_2_temp")


## Modificacion Columna: Fuel

In [118]:

# Limpiar y dividir en lista
df = df.with_columns([
    pl.col("Fuel")
    .str.replace_all(r"[()/,+]", " ")
    .str.replace_all(r"\s+", " ")
    .str.strip_chars()
    .str.split(" ")
    .alias("Fuel_list")
])

# Obtener el número máximo de elementos en Fuel_list
max_len = df.select(
    pl.col("Fuel_list").list.len().max().alias("max_len")
).to_numpy()[0][0]

# Convertir lista a struct
df = df.with_columns(
    pl.col("Fuel_list").list.to_struct(upper_bound=max_len).alias("Fuel_struct")
)

# Ahora unnest usando DataFrame.unnest()
df = df.unnest("Fuel_struct")

# Renombrar columnas generadas
df = df.rename({f"field_{i}": f"Fuel_{i+1}" for i in range(max_len)})

# Eliminar columna original Fuel y la lista
df = df.drop(["Fuel", "Fuel_list"])

##Mostrar la base de datos codificada y exportar el csv

In [119]:
# Mostrar primeras filas
df.head()

id,Brands,Model,Engines,Engine_capacity_in_cc,HorsePower_in_HP,HorsePower_in_HP_2,Max_speed_in_km/h,Time_to_100kmph_sec,Price_Min,Price_Max,Seats,Torque_in_Nm,Torque_in_Nm_2,Fuel_1,Fuel_2,Fuel_3
i64,str,str,str,i32,i32,i32,f32,f32,f32,f32,i32,i32,i32,str,str,str
0,"""Ferrari""","""SF90 STRADALE""","""V8""",3990,963,963,340.0,2.5,1.1e6,1.1e6,2,800,800,"""plug""","""in""","""hyrbrid"""
1,"""Rolls royce""","""PHANTOM""","""V12""",6749,563,563,250.0,5.3,460000.0,460000.0,5,900,900,"""Petrol""",null,null
2,"""Ford""","""KA+""","""1.2L Petrol""",1200,70,85,165.0,10.5,12000.0,15000.0,5,100,140,"""Petrol""",null,null
3,"""Mercedes""",""" GT 63 S""","""V8""",3982,630,630,250.0,3.2,161000.0,161000.0,4,900,900,"""Petrol""",null,null
4,"""Audi""","""AUDI R8 Gt""","""V10""",5204,602,602,320.0,3.6,253290.0,253290.0,2,560,560,"""Petrol""",null,null


In [ ]:
# Ruta de destino
output_path = r"C:\Users\Coder\Documents\REGRESIONES\F5ProjectV_ProblemaDeRegresion\data\dataset_modelado_optimizado.csv"

# Exportar CSV
df.write_csv(output_path)